In [1]:
import library as l

import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import datetime
import os

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

from scipy.stats import linregress

In [2]:
loadstring = []
loadstring.append('seed_1828_size_40000_topsize_300_date_2018-05-18')
loadstring.append('seed_16376_size_40000_topsize_300_date_2018-05-18')
loadstring.append('seed_23662_size_40000_topsize_300_date_2018-05-18')

loop = range(len(loadstring))

start_year = 2001
end_year = 2014

main = {}

for j, lstr in enumerate(loadstring):
    main[j] = {}

    main[j]['models'] = {}
    for i, model in enumerate(['/LR_' + str(start_year+x) + '.data' for x in range(end_year - start_year + 1)]):
        main[j]['models'][i+1] = l.load_file(lstr + model)

    main[j]['test_idx'] = {}
    for i, indx in enumerate(['/test_idx_' + str(start_year+x) + '.data' for x in range(end_year - start_year + 1)]):
        main[j]['test_idx'][i+1] = l.load_file(lstr + indx)

    main[j]['train_idx'] = {}
    for i, indx in enumerate(['/train_idx_' + str(start_year+x) + '.data' for x in range(end_year - start_year + 1)]):
        main[j]['train_idx'][i+1] = l.load_file(lstr + indx)

    main[j]['top_f'] = l.load_file(lstr + '/top_f.data')
    main[j]['bot_f'] = l.load_file(lstr + '/top_f.data')
    main[j]['CV'] = l.load_file(lstr + '/CV_2001.data')
    main[j]['indices'] = l.load_file(lstr + '/indices.data')
    main[j]['ratings'] = l.load_file(lstr + '/ratings.data')
    main[j]['seen'] = l.load_file(lstr + '/seen.data')
    main[j]['features'] = list(main[j]['CV'].get_feature_names())

In [3]:
for z in loop:
    main[z]['tops'] = {}
    for j in range(1,15):
        for i, (_,x) in enumerate(sorted(zip(main[z]['models'][j].coef_[0], main[z]['features']), reverse=True)):
            main[z]['tops'][x] = []
            if i >= 300:
                break
    for j in range(1,15):
        for i, (_,x) in enumerate(sorted(zip(main[z]['models'][j].coef_[0], main[z]['features']), reverse=True)):
            if not main[z]['tops'].get(x) is None:
                if i + 1 > 1500:
                    k = 1500
                else:
                    k = i + 1
                main[z]['tops'][x].append(k)

    main[z]['bots'] = {}
    for j in range(1,15):
        for i, (_,x) in enumerate(sorted(zip(main[z]['models'][j].coef_[0], main[z]['features']))):
            main[z]['bots'][x] = []
            if i >= 300:
                break
    for j in range(1,15):
        for i, (_,x) in enumerate(sorted(zip(main[z]['models'][j].coef_[0], main[z]['features']))):
            if not main[z]['bots'].get(x) is None:
                if i + 1 > 1500:
                    k = 1500
                else:
                    k = i + 1
                main[z]['bots'][x].append(k)

for z in loop:
    main[z]['top_slopes'] = []
    temp = []
    for key, values in main[z]['tops'].items():
        main[z]['top_slopes'].append([key, values])
        temp.append(sum(values))
    main[z]['top_slopes'] = [y for (x,y) in sorted(zip(temp, main[z]['top_slopes']))]
    
for z in loop:
    main[z]['bot_slopes'] = []
    temp = []
    for key, values in main[z]['bots'].items():
        main[z]['bot_slopes'].append([key, values])
        temp.append(sum(values))
    main[z]['bot_slopes'] = [y for (x,y) in sorted(zip(temp, main[z]['bot_slopes']))]
    
top_slopes = []

temp = []
for z in loop:
    temp.append(set([x[0] for x in main[z]['top_slopes']]))
temp = list(set.intersection(*temp))
indx = {}

for z in loop:
    for i, m in enumerate(main[z]['top_slopes']):
        if m[0] in temp:
            if z == 0:
                top_slopes.append([m[0], [m[1]]])
                indx[m[0]] = len(top_slopes)-1
            else:
                top_slopes[indx.get(m[0])][1].append(m[1])
temp = [sum(x[1][0] + x[1][1] + x[1][2]) for x in top_slopes]
top_slopes = [y for (x,y) in sorted(zip(temp, top_slopes))]
#print(top_slopes)

bot_slopes = []
temp = []
for z in loop:
    temp.append(set([x[0] for x in main[z]['bot_slopes']]))
temp = list(set.intersection(*temp))
indx = {}

for z in loop:
    for i, m in enumerate(main[z]['bot_slopes']):
        if m[0] in temp:
            if z == 0:
                bot_slopes.append([m[0], [m[1]]])
                indx[m[0]] = len(bot_slopes)-1
            else:
                bot_slopes[indx.get(m[0])][1].append(m[1])
temp = [sum(x[1][0] + x[1][1] + x[1][2]) for x in bot_slopes]
bot_slopes = [y for (x,y) in sorted(zip(temp, bot_slopes))]

In [5]:
main[0]['models'][1].coef_

array([[ 0.06041787,  0.01175008,  0.02809919, ..., -0.00825192,
        -0.02273517,  0.05189959]])